<a href="https://colab.research.google.com/github/SaeedArisha/Using_Pretrained_Models/blob/master/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Text classification using a pre-trained model from TensorHub

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [0]:
splits = ["train[:60%]", "train[-40%:]", "test"]
splits, info = tfds.load(name="imdb_reviews", with_info=True, split=splits, as_supervised=True)
train_data, validation_data, test_data = splits

In [0]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
num_classes = info.features['label'].num_classes

print('The Dataset has a total of:')
print('\u2022 {:,} classes'.format(num_classes))

print('\u2022 {:,} movie reviews for training'.format(num_train_examples))
print('\u2022 {:,} movie reviews for testing'.format(num_test_examples))

The Dataset has a total of:
• 2 classes
• 25,000 movie reviews for training
• 25,000 movie reviews for testing


In [0]:
class_names = ["negative", "positive"]

In [0]:
for review, label in train_data.take(1):
  review = review.numpy()
  label = label.numpy()
  print('\nMovie Review:\n\n', review)
  print('\nLabel:', class_names[label])


Movie Review:

 b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

Label: negative


In [0]:
# Loading Word Embeddings 

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype = tf.string, trainable=True)

In [0]:
#Building Pipeline
batch_size =  512

train_batches = train_data.shuffle(num_train_examples // 4).batch(batch_size).prefetch(1)
validation_batches = validation_data.batch(batch_size).prefetch(1)
test_batches = test_data.batch(batch_size)

In [0]:
#Building model over a predefined model 

model = tf.keras.Sequential([
                            hub_layer,
                            tf.keras.layers.Dense(16,activation="relu"),
                            tf.keras.layers.Dense(1, activation="sigmoid")
])

In [0]:
#Training the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

prediction  = model.fit(train_batches, epochs=20, validation_data=validation_batches, verbose=1 )

Epoch 1/20
30/30 [==============================] - 3s 116ms/step - loss: 0.8050 - accuracy: 0.5615 - val_loss: 0.6665 - val_accuracy: 0.6097
Epoch 2/20
30/30 [==============================] - 3s 108ms/step - loss: 0.6459 - accuracy: 0.6303 - val_loss: 0.6255 - val_accuracy: 0.6539
Epoch 3/20
30/30 [==============================] - 3s 108ms/step - loss: 0.5975 - accuracy: 0.6829 - val_loss: 0.5819 - val_accuracy: 0.6959
Epoch 4/20
30/30 [==============================] - 3s 108ms/step - loss: 0.5537 - accuracy: 0.7272 - val_loss: 0.5450 - val_accuracy: 0.7336
Epoch 5/20
30/30 [==============================] - 3s 111ms/step - loss: 0.5140 - accuracy: 0.7619 - val_loss: 0.5114 - val_accuracy: 0.7611
Epoch 6/20
30/30 [==============================] - 3s 110ms/step - loss: 0.4764 - accuracy: 0.7885 - val_loss: 0.4796 - val_accuracy: 0.7849
Epoch 7/20
30/30 [==============================] - 3s 108ms/step - loss: 0.4400 - accuracy: 0.8122 - val_loss: 0.4497 - val_accuracy: 0.8015
Epoch 

In [0]:
# Evaluating the model

score = model.evaluate(test_batches, verbose=1)
for metric, value in zip(model.metrics_names, score):
    print(metric + ': {:.3}'.format(value))

49/49 [==============================] - 3s 64ms/step - loss: 0.3239 - accuracy: 0.8642
loss: 0.324
accuracy: 0.864
